no sys args version, for debug purpose

Following block is a script for running KMC on API=1, Always run it see if it works after make modification on code.


In [7]:
#!/usr/bin/env python

from kmcpy.io import InputSet,load_occ
from kmcpy.kmc import KMC
import os
import sys

def main(api=1,**kwargs):
    os.chdir("../examples")
    inputset=InputSet.from_json("./test_input.json")

    print(inputset._parameters.keys())
    inputset.parameter_checker()

    if api==1:


        inputset.set_parameter("occ",load_occ(inputset._parameters["mc_results"],inputset._parameters["supercell_shape"],api=inputset.api))
        # step 1 initialize global occupation and conditions
        kmc = KMC()
        events_initialized = kmc.initialization(**inputset._parameters) # v in 10^13 hz

        # # step 2 compute the site kernal (used for kmc run)
        kmc.load_site_event_list(inputset._parameters["event_kernel"])

        # # step 3 run kmc
        kmc.run_from_database(events=events_initialized,**inputset._parameters)
    if api>1:
        raise NotImplementedError
    pass


if __name__ == "__main__":

    main()


dict_keys(['v', 'equ_pass', 'kmc_pass', 'supercell_shape', 'fitting_results', 'fitting_results_site', 'lce_fname', 'lce_site_fname', 'prim_fname', 'event_fname', 'event_kernel', 'mc_results', 'T', 'comp', 'structure_idx', 'api'])
Initializing kMC calculations with pirm.json at ./inputs/prim.json ...
Supercell Shape:
 [[2 0 0]
 [0 1 0]
 [0 0 1]]
Converting supercell ...
Loading fitting results ...
Loading fitting results (site energy) ...
Loading occupation: [-1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1 -1
 -1 -1 -1 -1]
Fitted time and error (LOOCV,RMS)
01/21/2022, 16:49:17 149.8671381416 29.1495050791
Fitted KECI and empty cluster
[10.5029085379, 4.0803809727, 0.0, 0.0, -11.2639080302, 0.0, -4.0578126572, 0.0, -7.2124075988, 0.0, 0.0, 0.0, 0.0] 339.1083600548
Fitted time and error (LOOCV,RMS) (site energy)
01/21/2022, 16:49:17 149.8671381416 29.1495050791
Fitted KECI and empty cluster (site energy)
[10.5029085379, 4.0803809727, 0.0, 0.0, -11.2639080302, 0.0, -4.

/userhome1/david/repo/kmcPy/kmcpy/tracker.py:93: RuntimeWarning: invalid value encountered in true_divide
  corr_factor = np.linalg.norm(self.displacement,axis=1)**2/(self.hop_counter*a**2)


18	1.385E-06	2.389E+01	1.352E-09	2.875E-10	1.092E-01	2.127E-01	2.257E-01	0.29	0.24
19	1.485E-06	2.409E+01	1.274E-09	2.704E-10	1.029E-01	2.123E-01	2.234E-01	0.29	0.24
20	1.568E-06	2.297E+01	1.257E-09	2.442E-10	1.016E-01	1.943E-01	2.622E-01	0.29	0.24
21	1.613E-06	3.054E+01	1.691E-09	3.156E-10	1.367E-01	1.866E-01	3.365E-01	0.29	0.24
22	1.658E-06	3.156E+01	1.649E-09	3.172E-10	1.333E-01	1.924E-01	3.172E-01	0.29	0.24
23	1.693E-06	3.156E+01	1.615E-09	3.106E-10	1.305E-01	1.924E-01	3.168E-01	0.29	0.24
24	1.766E-06	3.771E+01	2.290E-09	3.559E-10	1.851E-01	1.554E-01	2.691E-01	0.29	0.24
25	1.819E-06	4.013E+01	2.359E-09	3.677E-10	1.906E-01	1.559E-01	2.824E-01	0.29	0.24
26	1.940E-06	3.553E+01	1.915E-09	3.053E-10	1.548E-01	1.594E-01	2.578E-01	0.29	0.24
27	2.012E-06	3.546E+01	1.722E-09	2.937E-10	1.392E-01	1.705E-01	2.558E-01	0.29	0.24
28	2.124E-06	3.771E+01	1.904E-09	2.959E-10	1.539E-01	1.554E-01	2.573E-01	0.29	0.24
29	2.168E-06	3.546E+01	1.598E-09	2.725E-10	1.292E-01	1.705E-01	2.526E-01	0.29	0.24
30	2

following block is for generating the local cluster expansion model. See the Table S3 in support information of nasicon kmc as reference

The local cluster expansion is for generating the lce.json

In [3]:
from kmcpy.model import LocalClusterExpansion

a=LocalClusterExpansion()
a.initialization()
a.to_json("testlce.json")

FileNotFoundError: [Errno 2] No such file or directory: 'EntryWithCollCode15546_Na4Zr2Si3O12_573K.cif'

the kmc.event_generator is for generating the event_kernal.csv

In [4]:
from kmcpy.event_generator import generate_events

generate_events(r"../examples/inputs/prim.json",[2,1,1],"./testevent_kernal.csv")

Generating events ...
Initializing model with pirm.json at ../examples/inputs/prim.json ...
Supercell Shape:
 [[2 0 0]
 [0 1 0]
 [0 0 1]]
Converting supercell ...
Looking for all possible swaps with ncpus = 40  (this might take a while) ...
Saving: ./testevent_kernal.csv
Generating event kernal ...
Saving into: event_kernal.csv


Api version 2 initiated
aiming for more flexible inputs.
Following block is the main KMC function
DO make sure that they provide similar result as in API=1

In [6]:
#!/usr/bin/env python

from kmcpy.io import InputSet,load_occ
from kmcpy.kmc import KMC
import os
import sys

def main(api=2,**kwargs):
    inputset=InputSet.from_json("test_input_v2.json",api=2)

    print(inputset._parameters.keys())
    inputset.parameter_checker()

    inputset.set_parameter("occ",load_occ(fanme=inputset._parameters["mc_results"],shape=inputset._parameters["supercell_shape"],select_sites=inputset._parameters["select_sites"],api=inputset.api,verbose=True))
    
    
    

    # step 1 initialize global occupation and conditions
    
    
    
    kmc = KMC(api=api)
    events_initialized = kmc.initialization(**inputset._parameters) # v in 10^13 hz

    # # step 2 compute the site kernal (used for kmc run)
    kmc.load_site_event_list(inputset._parameters["event_kernel"])

    # # step 3 run kmc
    kmc.run_from_database(events=events_initialized,**inputset._parameters)


if __name__ == "__main__":

    main()

dict_keys(['v', 'equ_pass', 'kmc_pass', 'supercell_shape', 'fitting_results', 'fitting_results_site', 'lce_fname', 'lce_site_fname', 'prim_fname', 'event_fname', 'event_kernel', 'mc_results', 'select_sites', 'T', 'comp', 'structure_idx', 'api', 'immutable_sites', 'verbose', 'random_seed', 'use_numpy_random_kernel'])
verbose output is enabled, this is function io.load_acc from Api2
selected sites are [0, 1, 2, 3, 4, 5, 6, 7, 12, 13, 14, 15, 16, 17]
converting the occupation raw data to dimension: (42, 2, 1, 1)
occupation_chebyshev: [-1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1 -1
 -1 -1 -1 -1]
Initializing kMC calculations with pirm.json at ./inputs/prim.json ...
Supercell Shape:
 [[2 0 0]
 [0 1 0]
 [0 0 1]]
Converting to the supercell ...
removing the immutable sites: ['Zr', 'O']
Loading fitting results E_kra...
Loading fitting results (site energy or E_end) ...
Loading occupation: [-1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1 -1
 -1 -1 -

/userhome1/david/repo/kmcPy/kmcpy/tracker.py:93: RuntimeWarning: invalid value encountered in true_divide
  corr_factor = np.linalg.norm(self.displacement,axis=1)**2/(self.hop_counter*a**2)


15	4.119E-06	2.655E+01	7.079E-11	1.074E-10	5.721E-03	1.517E+00	1.950E-01	0.29	0.24
16	4.190E-06	2.269E+01	7.218E-11	9.027E-11	5.834E-03	1.251E+00	1.685E-01	0.29	0.24
17	4.386E-06	2.861E+01	1.086E-10	1.087E-10	8.773E-03	1.001E+00	2.140E-01	0.29	0.24
18	4.465E-06	3.647E+01	1.537E-10	1.361E-10	1.242E-02	8.858E-01	2.667E-01	0.29	0.24
19	4.518E-06	3.926E+01	7.811E-11	1.449E-10	6.312E-03	1.855E+00	2.650E-01	0.29	0.24
20	4.552E-06	3.845E+01	6.519E-11	1.408E-10	5.269E-03	2.160E+00	2.608E-01	0.29	0.24
21	4.581E-06	3.764E+01	1.583E-10	1.369E-10	1.279E-02	8.649E-01	2.488E-01	0.29	0.24
22	4.660E-06	4.423E+01	6.355E-11	1.582E-10	5.136E-03	2.489E+00	2.574E-01	0.29	0.24
23	4.816E-06	5.154E+01	8.200E-11	1.783E-10	6.627E-03	2.175E+00	2.629E-01	0.29	0.24
24	4.995E-06	5.114E+01	1.326E-10	1.706E-10	1.072E-02	1.287E+00	2.727E-01	0.29	0.24
25	5.045E-06	5.545E+01	9.717E-11	1.832E-10	7.853E-03	1.885E+00	2.766E-01	0.29	0.24
26	5.376E-06	5.602E+01	1.279E-10	1.737E-10	1.034E-02	1.358E+00	2.659E-01	0.29	0.24
27	5